In [12]:
from datasets import load_dataset

ds = load_dataset("MohamedSaeed-dev/python-text-to-code")

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--MohamedSaeed-dev--python-text-to-code. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 887/887 [00:00<00:00, 78824.26 exampl

In [4]:
# Check keys (splits)
print(ds)


DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 463
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 51
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 60
    })
})


In [2]:
import json

files = [
    ("train.json", "train.jsonl"),
    ("test.json", "test.jsonl"),
    ("valid.json", "valid.jsonl"),
]

for input_file, output_file in files:
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Just copy line-by-line if they are already JSON objects
    with open(output_file, "w", encoding="utf-8") as f:
        for line in lines:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    json_obj = json.loads(line)  # Validate JSON
                    f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
                except json.JSONDecodeError as e:
                    print(f"❌ Error in line: {line[:80]}... → {e}")

    print(f"✅ Processed {input_file} → {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'train.json'

In [1]:
import json
from deep_translator import GoogleTranslator
from tqdm import tqdm

# === Step 1: Translate text ===

# File paths
input_file = "Python Train.jsonl"            # English input
translated_file = "Python_Train_ms.jsonl"    # Intermediate file with both English & Malay
final_output_file = "Python_Train_Data.jsonl"  # Final output: only Malay prompt + code

# Field names
source_field = "text"
translated_field = "text_ms"

translated_data = []

print("🔄 Translating English prompts to Malay...")
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in tqdm(lines):
    item = json.loads(line)
    english_text = item.get(source_field, "")

    try:
        malay_text = GoogleTranslator(source="en", target="ms").translate(english_text)
    except Exception as e:
        print(f"⚠️ Translation error: {e}")
        malay_text = "[TRANSLATION_FAILED]"

    item[translated_field] = malay_text
    translated_data.append(item)

# Save intermediate JSONL with English + Malay
with open(translated_file, "w", encoding="utf-8") as f:
    for item in translated_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Translated data saved to: {translated_file}")

# === Step 2: Extract only Malay + Code ===

cleaned_data = []
for item in translated_data:
    cleaned_item = {
        "text": item.get("text_ms", "[NO_TRANSLATION]"),
        "code": item.get("code", "")
    }
    cleaned_data.append(cleaned_item)

# Save final simplified JSONL
with open(final_output_file, "w", encoding="utf-8") as f:
    for item in cleaned_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Final cleaned dataset saved to: {final_output_file}")

🔄 Translating English prompts to Malay...


 48%|████▊     | 4413/9263 [1:37:19<9:19:44,  6.92s/it]

⚠️ Translation error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 9263/9263 [3:21:00<00:00,  1.30s/it]  


✅ Translated data saved to: Python_Train_ms.jsonl
✅ Final cleaned dataset saved to: Python_Train_Data.jsonl


In [ ]:
from deep_translator import ChatGptTranslator

text = 'create the link list from the given array'
translated = ChatGptTranslator(api_key= "", model="", target='zh-CN', source='en').translate(text)
print("Original Text:", text)
print("Translated text:", translated)

ApiKeyException: None --> 
You have to pass your api_key!
You can do this by passing the key as a parameter/argument to the translator class
or by setting the environment variable OPEN_API_KEY

Example: export OPEN_API_KEY="your_api_key"
